In [1]:
import json
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, DataFrameWriter, DataFrameReader
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
def get_config():
    with open("secret.json", "r") as f:
        jsonstr = f.read()
        conf = json.loads(jsonstr)
        return conf

def get_spark_conf(config):
    '''set config'''
    conf = SparkConf()
    conf.setAppName('abb_t')
    conf.set('spark.master', config["spark"]["master_url"])
    return conf

def get_pg_props(config):
    '''set psql properties'''
    props = {
        "user": config["postgres"]["user"],
        "password": config["postgres"]["password"],
        "driver": "org.postgresql.Driver",
    }
    return props

In [13]:
def getdf(sql_context, config): 
    '''filter abb dataset'''
    df = sql_context \
        .read.format('csv').options(header='true') \
        .load(config["s3"]["abb_toronto_listings"])
    return df

In [5]:
def write_to_pg(toronto_listings, config):
    '''write to psql'''
    url = "jdbc:postgresql://10.0.0.14/xyn"
    table = 'abb'
    mode = 'overwrite'
    props = get_pg_props(config)
    toronto_listings.write.jdbc(url, table, mode, props)

In [6]:
config = get_config()
spark_conf = get_spark_conf(config)
sc = SparkContext(conf=spark_conf)

In [7]:
sql_context = SQLContext(sc)

In [14]:
df = getdf(sql_context, config)

In [15]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: string (nullable = true)



In [24]:
toronto_listings = df.withColumn("latitude", df["latitude"].cast(DoubleType())) \
                    .withColumn("longitude", df["longitude"].cast(DoubleType())) \
                    .withColumn("price", df["price"].cast(IntegerType())) \
                    .withColumn("number_of_reviews", df["number_of_reviews"].cast(IntegerType()))\
                    ["latitude", "longitude", "price", "number_of_reviews"]

In [25]:
toronto_listings.printSchema()

root
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- price: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)



In [27]:
toronto_listings.head()

Row(latitude=43.646167661556824, longitude=-79.42451012783086, price=469, number_of_reviews=7)

In [28]:
toronto_listings.describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|          latitude|         longitude|             price| number_of_reviews|
+-------+------------------+------------------+------------------+------------------+
|  count|             19250|             19208|             19250|             19208|
|   mean| 43.38517565754363|-79.37134977935965|139.52005194805196|23.738025822573928|
| stddev| 6.009661132118586|1.7767816752807741|210.53457865340496| 42.61145776544502|
|    min|-79.57713012060132|-79.62345558679347|                 0|                 0|
|    max| 43.83946750721422|43.704626131941126|             13315|               643|
+-------+------------------+------------------+------------------+------------------+



In [29]:
write_to_pg(toronto_listings, config)